<a href="https://colab.research.google.com/github/stmeinert/Recolorization_IANN/blob/main/calculate_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook zum Bestimmen der Metriken



In [32]:
#import models from github
!git clone https://github.com/stmeinert/Recolorization_IANN.git

import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm
!pip install tensorflow-io
!pip install deepface
from deepface import DeepFace
import tensorflow_io as tfio
import time
import os
import statistics
import numpy as np
from sklearn import metrics
from collections import defaultdict
import pickle
import matplotlib.pyplot as plt
import csv
import sys
if not "/content/Recolorization_IANN" in sys.path:
    sys.path.append("/content/Recolorization_IANN")
from src.iizuka.iizuka_recolorization_model import IizukaRecolorizationModel
from src.zhang.zhang import CIC
from src.zhang_prob.zhang_prob import CIC_Prob

from src.data_util.data_pipeline_util import unzip_and_load_ds


tf.keras.backend.clear_session()

fatal: destination path 'Recolorization_IANN' already exists and is not an empty directory.


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
#choose models

#model = IizukaRecolorizationModel(BATCH_SIZE)
model = CIC()
#model = CIC_Prob()

# DS_NAME = "celeb_data_set_preprocessed_part_0_3"
DS_NAME = "celeb_data_set_unbatch_30000"

ZIP_DS_PATH = '/content/drive/MyDrive/zip_file/' + DS_NAME + '.zip'
EXTRACT_DS_PATH = '/content/current/Dataset'


# size of training, test and validation sets
BATCH_SIZE = 128
TRAIN_IMAGES = 25000
TEST_IMAGES = BATCH_SIZE
VAL_IMAGES = 2500

EPOCHS = 21


MODEL_SAVE_LOCATION = "/content/drive/MyDrive/checkpoints/24_03"
LOG_SAVE_LOCATION = "/content/drive/MyDrive/logs/"
DATA_SAVE_LOCATION = "/content/drive/MyDrive/save_data/25_03"

In [ ]:
# import/download der checkpoints bzw. genauer bestimmen welche Nummer

In [28]:
#load dataset for validation calculation
@tf.function
def prepare_train_dataset(image_ds):
    image_ds = image_ds.cache()
    image_ds = image_ds.shuffle(1000)
    image_ds = image_ds.batch(BATCH_SIZE, drop_remainder=True)
    image_ds = image_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return image_ds

@tf.function
def prepare_test_dataset(image_ds):
    image_ds = image_ds.cache()
    image_ds = image_ds.batch(BATCH_SIZE, drop_remainder=True)
    image_ds = image_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return image_ds

### get Dataset in place

ds = unzip_and_load_ds(DS_NAME, EXTRACT_DS_PATH, ZIP_DS_PATH)
train_ds = ds.take(TRAIN_IMAGES)
train_ds = train_ds.apply(prepare_train_dataset)
val_ds = ds.skip(TRAIN_IMAGES).take(VAL_IMAGES)
val_ds = val_ds.apply(prepare_train_dataset)
test_ds = ds.skip(TRAIN_IMAGES+VAL_IMAGES).take(TEST_IMAGES)
test_ds = test_ds.apply(prepare_test_dataset)

In [29]:
# methods for the metrics

# append to a list
# the root mean squared error
# prediction is a tensor of shape (1,image_width, image_height,2) <-- (a,b)
# target is a tensor of shape ((1,image_width, image_height,3) <-- (l,a,b))
# Important! normalization needed 
def append_calculate_rmse(prediction,target,result_list):
    pa = prediction.numpy()
    ta = target.numpy()
    for b in range(ta.shape[0]):
      for i in range(ta.shape[1]):
        for j in range(ta.shape[2]):
            result_list.append(np.sqrt(0.5* ((ta[b][i][j][1]-pa[i][j][1])**2 + (ta[b][i][j][2]-pa[i][j][2])**2 ))/255)


# creates data for the cumulative histogram to plot it
# input_list root mean squared error values of each pixel
# x_values of the interval to check the amount in one bin
def create_plot_data_from_list(input_list,x_values):
    input_array = np.array(input_list)
    y_percent = []
    for i in range(len(x_values)):
        y_percent.append((input_array <= x_values[i]).sum()/input_array.size)
    return y_percent

Main

In [36]:
# calculate metrics
print("################ GPU in use: ################")
!nvidia-smi -L
print("#############################################")


ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer=model.optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, MODEL_SAVE_LOCATION, max_to_keep=3)

ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
else:
    raise ValueError('Could not restore the model form the given checkpoint')



#train_log_path = f"{LOG_SAVE_LOCATION}/train"
#val_log_path = f"{LOG_SAVE_LOCATION}/val"
#img_test_log_path = f"{LOG_SAVE_LOCATION}/img_test"
# log writer for training metrics


metrics = ["cosine", "euclidean", "euclidean_l2"]
# input target dict
metrics_input_target_dict = defaultdict(list)
metrics_input_target_dict["metrics_list_cosine_input_target"]
metrics_input_target_dict["metrics_list_euclidean_input_target"]
metrics_input_target_dict["metrics_list_euclidean_l2_input_target"]
# prediction target dict
metrics_prediction_target_dict = defaultdict(list)
metrics_prediction_target_dict["metrics_list_cosine_prediction_target"]
metrics_prediction_target_dict["metrics_list_euclidean_prediction_target"]
metrics_prediction_target_dict["metrics_list_euclidean_l2_prediction_target"]

#face verification
#result = DeepFace.verify(img1_path = "img1.jpg", img2_path = "img2.jpg", distance_metric = metrics[1])
counter = 0
rmse_list = []

print("Calculating loss using the deepface package on the test_ds")
for input, target in tqdm.notebook.tqdm(test_ds.take(1),position=0, leave=True):
      prediction = model(input)
      # get l channel, target should be in shape (SIZE, SIZE, lab)
      l = tf.slice(target, begin=[0,0,0,0], size=[-1,-1,-1,1])
      prediction = tf.concat([l, prediction], axis=-1) # should be concatenating along last dimension
      prediction = tfio.experimental.color.lab_to_rgb(prediction)
      target = tfio.experimental.color.lab_to_rgb(target)
      input = (input+1)*50
      two_zeros = tf.zeros((BATCH_SIZE,128,128,2))
      input = tf.concat([input, two_zeros], -1)
      # calculating rmse for input target
      append_calculate_rmse(input,target,rmse_list)
      input_rgb =  tfio.experimental.color.lab_to_rgb(input)
      for b in range(input_rgb.shape[0]):
        # TODO iterate over BATCH
        #print('target')
        #plt.imshow(target.numpy()[0][:][:][:].astype("float32"))
        plt.imsave("target.jpg",target.numpy()[b][:][:][:].astype("float32"))
        #print('prediction')
        #plt.imshow(prediction.numpy()[0][:][:][:].astype("float32"))
        # just l channel
        #print('input')
        #plt.imshow(input.numpy()[0][:][:][:].astype("float32"))
        plt.imsave("input.jpg",input_rgb.numpy()[b][:][:][:].astype("float32"))
        #plt.show()
        print(counter)
        counter += 1
        #using deepface to calculate metrics
        for i in range(len(metrics)):
          try: 
            result = DeepFace.verify(img1_path = "input.jpg", img2_path = "target.jpg", distance_metric = metrics[i],enforce_detection=False)
          except Exception as e:
            print(e)
            plt.imshow(input_rgb.numpy()[b][:][:][:].astype("float32"))
            plt.show()
            plt.imshow(target.numpy()[b][:][:][:].astype("float32"))
            plt.show()
          else:
            metrics_input_target_dict['metrics_list_'+metrics[i]+'_input_target'].append(result['distance'])
          #TODO
          #insert second list here
    

#for i in range(len(metrics)):
  #for j in range(len(metrics_list_input_target[i])):
#  print(statistics.mean(metrics_list_input_target[i]))
print(metrics_input_target_dict.keys())
for key in  metrics_input_target_dict:
  print(key)
  if len(metrics_input_target_dict[key]) > 0:
    print(statistics.mean(metrics_input_target_dict[key]))

#save dict 
with open(DATA_SAVE_LOCATION+'/metrics_input_target_dict.pickle', 'wb') as f:
    pickle.dump(metrics_input_target_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(DATA_SAVE_LOCATION+'/metrics_input_target_dict.pickle','rb') as e:
     testdict = pickle.load(e)
print(testdict.keys())
x_values = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
x_values_array = np.array(x_values)
np.save(DATA_SAVE_LOCATION+'/x_values.npy', x_values_array)

plot_data_array = np.array(create_plot_data_from_list(rmse_list,x_values))
np.save(DATA_SAVE_LOCATION+'/histogram_data.npy', plot_data_array)

# TODO do something with the metrics





################ GPU in use: ################
GPU 0: Tesla T4 (UUID: GPU-ea42aeb1-3bcb-5e62-1560-b5121de8eccc)
#############################################
Restored from /content/drive/MyDrive/checkpoints/24_03/ckpt-21
Calculating loss using the deepface package on the test_ds


  0%|          | 0/1 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
dict_keys(['metrics_list_cosine_input_target', 'metrics_list_euclidean_input_target', 'metrics_list_euclidean_l2_input_target'])
metrics_list_cosine_input_target
0.025327351072245007
metrics_list_euclidean_input_target
0.16262548002510643
metrics_list_euclidean_l2_input_target
0.2014396125395276
dict_keys(['metrics_list_cosine_input_target', 'metrics_list_euclidean_input_target', 'metrics_list_euclidean_l2_input_target'])


In [ ]:
# visualierung der metriken